### Notebook to compute travel demand with Gravitational Model and travel weights using LOGIT model

#### Initial data

In [4]:
# number of travels passed as a matrix with produced travels on lines and attracted ones on columns
travs = [[40, 110, 150],
         [50, 20, 30],
         [110, 30, 10]]

# the matching friction factors, same arrangement

ffs = [[0.753, 1.597, 0.753],
       [0.987, 0.753, 0.765],
       [1.597, 0.765, 0.753]]

# neutral calibration coefficients

k_ij0 = [[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]]

# auto travels cost

tca = [[0.5, 1, 1.4],
       [1.2, 0.8, 1.2],
       [1.7, 1.5, 0.7]]

# transit travels cost

tct = [[1, 1.5, 2],
       [1.8, 1.2, 1.9],
       [1.7, 1.5, 0.7]]

# auto travels duration

tda = [[3, 12, 7],
       [13, 3, 19],
       [9, 16, 4]]

# transit travels duration

tdt = [[15, 5, 12],
       [15, 6, 26],
       [20, 21, 8]]

# the future friction factors

ffs_f = [[0.753, 0.987, 1.597],
         [0.987, 0.753, 0.765],
         [1.597, 0.765, 0.753]]

# the future produced travels

P_is = [750, 580, 480]

# the future attracted travels

A_js = [722, 786, 302]

#### Class to compute the travels using Gravitational Model

In [71]:
class GravitMod:
    def __init__(self, travs, ffs, k_ijs, P_is, A_js):
        self.travs = travs
        self.ffs = ffs
        self.k_ijs = k_ijs
        self.P_is = P_is
        self.A_js = A_js

    
    def transp_mat(mat):
        """
        Method to transpose a squared matrix.
        Takes as input the matrix, returns the transpose, same shape.
        """
        # check for same number of rows and columns (square matrix)
        if(len(mat) != len(mat[0])):
            print("The matrix is not squared. Please provide a squared matrix.")
            exit()

        # transpose the matrix
        transp = list(zip(*mat))

        return [list(sublist) for sublist in transp]

    
    def sum_mat(mat):
        """
        Method to compute the sum of rows of a squared matrix.
        Takes as input the matrix, returns the sum of rows.
        """

        if(len(mat) != len(mat[0])):
            print("The matrix is not squared. Please provide a squared matrix.")
            exit()

        sum_mat = []    # empty list to store the sums

        for row in mat:
            sum_mat.append(sum(row))

        return sum_mat
    
    
    def formula(s_Pi, s_Aj, ffs, k_ijs):
        """
        Method to apply the Gravitational Model formula.
        Takes as inputs the sum of produced, respectively attracted travels,
        the matrix of friction factors, and the matrix of calibration coefficients.
        Returns the squared matrix of computed travels.
        """

        gvals_init = []    # to store computed values
        for i in range(len(travs)):
            pdsum = 0
            for j1, j2 in zip(s_Aj, ffs[i]):
                pdsum = pdsum + j1 * j2
            for k1 in range(len(ffs[i])):
                gvals_init.append((s_Pi[i] * ffs[i][k1] * s_Aj[k1] * k_ijs[i][k1] / pdsum))

        return gvals_init
        
    
    def rnd(mat):
        """
        Method to round the travels raw computed with Gravitational Model.
        Takes as input the single row matrix, returns rounded values, same shape.
        """
        mat_r = []
        for item in mat:
            mat_r.append(round(item))

        return mat_r
    
    def mat_grp(mat):
        """
        Method to group the rounded single row matrix into a squared one.
        Takes as input the single row matrix of rounded travels and
        returns the squared matrix of travels obtained with
        Gravitational Model.
        """
        return [mat[i:i + 3] for i in range(0, len(mat), 3)]
    
    def gravmod_init(travs, ffs, k_ijs):
        """
        Method to compute gravitational model values in order to determine the
        calibration factors.
        Takes as input the travels, friction factors and calibration
        coefficients matrices.
        Returns a matrix with the computed travels.
        """
    
        # check if the matrices have the same shape
        if(len(travs) != len(ffs) or (len(travs) != len(k_ijs))):
            print("The matrices doesn't match. Please fix it.")
            exit()
    
        # transpose the travels and friction factors matrices
        travs_t = GravitMod.transp_mat(travs)
        ffs_t = GravitMod.transp_mat(ffs)
    
        # get attracted travels sums
        s_Aj = GravitMod.sum_mat(travs_t)

        # get produced travels sums
        s_Pi = GravitMod.sum_mat(travs)
    
        # compute travels with gravitational model
        gvals_init = GravitMod.formula(s_Pi, s_Aj, ffs, k_ijs)

        # round the computed single row matrix
        gvals_init_r = GravitMod.rnd(gvals_init)
    
        # group flatten list 'gvals_init_r' as a matrix
        gvals_init_m = GravitMod.mat_grp(gvals_init_r)
        
        return gvals_init_m

In [73]:
# compute and print travels using Gravitational Model on historical
# data for later calibration coefficients dete
gvalsr = GravitMod.gravmod_init(travs, ffs, k_ij0)
print("Returned, ", gvalsr)

Returned,  [[82, 140, 78], [43, 26, 31], [82, 31, 37]]
